In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
test_files = ['../data/lgb.csv.gz',
             '../data/xgb.csv.gz',
             '../data/nn.csv.gz']
val_files = ['../data/lgb_val.pickle',
            '../data/xgb_val.pickle',
            '../data/nn_val.pickle']

In [3]:
dfs_test = [pd.read_csv(f) for f in test_files]

In [4]:
import pickle
dfs_val = []
for val_file in val_files:
    with open(val_file, 'rb') as handle:
        df_val = pickle.load(handle)
        dfs_val.append(df_val)

In [14]:
L, R = 0, 3000000
for df_val in dfs_val:
    t = df_val[L: R]
    print(roc_auc_score(t.y, t.y_pred))

0.9692657721875765
0.9689437177046202
0.9575134586788727


In [15]:
for i in range(11):
    weights = [7, 3, i]
    t = dfs_val[0][L: R].y_pred * weights[0]
    for i in range(1, len(dfs_val)):
        t += dfs_val[1][L: R].y_pred * weights[i]
    print(weights, roc_auc_score(dfs_val[0][L: R].y, t))

[7, 3, 0] 0.9696351203854966
[7, 3, 1] 0.9696527589986236
[7, 3, 2] 0.9696568177004271
[7, 3, 3] 0.9696538100643365
[7, 3, 4] 0.9696463844178129
[7, 3, 5] 0.969635357709042
[7, 3, 6] 0.9696230982862563
[7, 3, 7] 0.9696098971933562
[7, 3, 8] 0.9695967551772063
[7, 3, 9] 0.9695829875965214
[7, 3, 10] 0.9695693368791111


In [16]:
weights = [7, 3, 1]
weights = weights / np.sum(weights)

In [17]:
df = pd.DataFrame({'click_id': dfs_test[0].click_id.astype(int).values, 'is_attributed': np.zeros(dfs_test[0].shape[0])})

In [18]:
for tdf, weight in zip(dfs_test, weights):
    df['is_attributed'] += tdf['is_attributed'] * weight

In [19]:
df.to_csv('../data/blend.csv.gz', compression='gzip', index=False)

In [21]:
df[df.is_attributed > 1]

,click_id,is_attributed
